In [1]:
import pandas as pd
from PIL import Image
import os
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score
# from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# first, preprocess data

df = pd.read_csv("./metadata/training_mixed_frames_labels.csv", header=0)

print(df.shape[0])

print(df.iloc[0])

indicesToKeep = []

timeSinceLastAddition = 0

# so i kinda done goofed and deleted some images from the dataset, issue is now though that some things are misalinged where it follows
# up a series of even images by odd images, so have to kinda parse through them. How we select them is find where the image changes, select
# a bit slightly before, and kinda do it that way until we win.
timeAlters = [0, 7022, 8201, 9751, 15568, 18574, 19235, 22083, 22384, 23748, 28061, 37895, 38118, 38286, 41301, df.shape[0] - 1]


for i in range(0, len(timeAlters) - 1, 2):
    for j in range(timeAlters[i], timeAlters[i + 1] + 1, 2):
        # first condition checks if we're at teh end, 2nd checks for a check, 3rd checks if too mcu htime has passed and its aligned
        if(j == timeAlters[i + 1] or df.iloc[j]["label"] != df.iloc[j + 1]["label"] or (timeSinceLastAddition > 14 and j % 2 == timeAlters[i] % 2)):
            indicesToKeep.append(j - 4)
            indicesToKeep.append(j - 8)
            timeSinceLastAddition = 0
        else:
            timeSinceLastAddition += 1

optimizedDf = df.iloc[indicesToKeep].reset_index(drop=True)

class0Data = optimizedDf[(optimizedDf["label"] == 0) | (optimizedDf["label"] == 7)]

class1Data = optimizedDf[(optimizedDf["label"] == 1) | (optimizedDf["label"] == 2) | (optimizedDf["label"] == 3) | (optimizedDf["label"] == 4) | (optimizedDf["label"] == 17)]


print(class0Data.shape[0])
print(class1Data.shape[0])

class0Data["label"] = 0
class0Data = class0Data.sample(class1Data.shape[0], random_state=42)

class1Data["label"] = 1

print(class0Data.shape[0])
print(class1Data.shape[0])

finalDf = pd.concat([class0Data, class1Data], ignore_index=True)

print(finalDf.shape[0])


# stuff for deleteion, don't touch normally
# print(df.shape[0])
# df_filtered = df.drop(finalDf["frame_id"].tolist())
# print(df_filtered.shape[0])
# for frame in range(df_filtered.shape[0]):
#     currentImageIndex = df_filtered.iloc[frame]["frame_id"]
#     imagePath = f"./datasetNew/{currentImageIndex:06d}_512_512.jpg"
#     if os.path.exists(imagePath):
#         os.remove(imagePath)
#         print(f"{imagePath} deleted.")

# # list all items in the folder
# children = os.listdir("./datasetNew")

# print(f"Number of items in the folder: {len(children)}")

144603
frame_id    0
label       1
Name: 0, dtype: int64
3440
1356
1356
1356
2712


C:\Users\mahd\AppData\Local\Temp\ipykernel_9788\5582965.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class0Data["label"] = 0
C:\Users\mahd\AppData\Local\Temp\ipykernel_9788\5582965.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class1Data["label"] = 1


In [3]:
images = np.zeros((finalDf.shape[0], 96, 96), dtype=np.float32)
print(images.shape)  # (100, 512, 512)

# def returnProcessedImage(dfIndex, pictureNumber):
#     currentImage = Image.open(f"datasetOg/{pictureNumber:06d}_512_512.jpg")
#     # convert this to 96 x 96 and grayscale
#     processedImage = currentImage.resize([96, 96], Image.Resampling.LANCZOS).convert("L")
#     normalizedNpArrImage = np.array(processedImage, dtype=np.float32) / 255.0

#     return (dfIndex, normalizedNpArrImage)

# you guys ever just spend a good amount of time  on something only to realize it's all useless? yeah that's what happened right below

# currentDfIndex = 0
# batchSize = 10
# while(currentDfIndex < finalDf.shape[0]):
#     with ThreadPoolExecutor(max_workers=10) as executor:
#             futures = []
#             for i in range(0, min(finalDf.shape[0], currentDfIndex + batchSize), 1):
#                  dfIndex = currentDfIndex + i
#                  futures.append(executor.submit(returnProcessedImage, dfIndex, finalDf.iloc[dfIndex]["frame_id"]))
#             for future in as_completed(futures):
#                  result = future.result()
#                  images[result[0]] = result[1]
#     currentDfIndex += batchSize
#     if(currentDfIndex % 2000 == 0):
#         print(f"Made it to index {currentDfIndex}!")

for i in range(0, finalDf.shape[0], 1):
    actualPicIndex = finalDf.iloc[i]["frame_id"]
    currentImage = Image.open(f"datasetOg/{actualPicIndex:06d}_512_512.jpg")
    # convert this to 96 x 96 and grayscale
    processedImage = currentImage.resize([96, 96], Image.Resampling.LANCZOS).convert("L")
    normalizedNpArrImage = np.array(processedImage, dtype=np.float32) / 255.0

    images[i] = normalizedNpArrImage

(2712, 96, 96)


In [4]:
labels = finalDf["label"].to_numpy()

# now shuffle them when passing in
randomIndexOrder = np.random.permutation(labels.shape[0])

images = images[randomIndexOrder]
labels = labels[randomIndexOrder]

# also must add an extra dimension
if images.ndim == 3:  # shape = (num_samples, 96, 96)
    images = images[..., np.newaxis]  # add channel dim
elif images.ndim != 4:
    images = np.squeeze(images, axis=-1)

# now make a training and test set
testPercentage = 0.95
cutoffPoint = int(testPercentage * labels.shape[0])

trainingXSet = images[0:cutoffPoint]
trainingYSet = labels[0:cutoffPoint]

testingXSet = images[cutoffPoint:]
testingYSet = labels[cutoffPoint:]



In [5]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
model = Sequential([
    # 96 x 96
    Conv2D(16, (3, 3), activation='relu', padding="same", input_shape=(96, 96, 1)),
    # becomes 48 x 48 after
    MaxPooling2D((2, 2)),

    # 
    Conv2D(32, (3, 3), padding="same", activation='relu'),
    GlobalAveragePooling2D(),

    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # for binary classification
])

# tensorflow is werid, so apparently you gotta compile it first? interesting
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', 'AUC', 'Precision', 'Recall']
)

# history = model.fit(
#     trainingXSet, trainingYSet,
#     batch_size=16,
#     epochs=30
# )

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

datagen.fit(trainingXSet)  # if trainingXSet is a NumPy array

model.fit(datagen.flow(trainingXSet, trainingYSet, batch_size=16),
          epochs=30)


predictions = model.predict(testingXSet) 

# apparently this just works
predicted_classes = (predictions > 0.5).astype(int)

Epoch 1/30


c:\Users\mahd\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\mahd\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


161/161 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - AUC: 0.4759 - Precision: 0.5014 - Recall: 0.7591 - accuracy: 0.4910 - loss: 0.6938
Epoch 2/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - AUC: 0.4883 - Precision: 0.1085 - Recall: 0.0256 - accuracy: 0.5060 - loss: 0.6932  
Epoch 3/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - AUC: 0.4681 - Precision: 0.5035 - Recall: 0.8432 - accuracy: 0.4974 - loss: 0.6945
Epoch 4/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - AUC: 0.4699 - Precision: 0.4674 - Recall: 0.6267 - accuracy: 0.4794 - loss: 0.6934
Epoch 5/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - AUC: 0.4930 - Precision: 0.2645 - Recall: 0.1575 - accuracy: 0.5147 - loss: 0.6930
Epoch 6/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - AUC: 0.4934 - Precision: 0.5125 - Recall: 0.7954 - accuracy: 0.5109 - loss: 0.6930
Epoch 7/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - AUC: 0.4900 - Precision: 0.4378 - Recall: 0.1262 - accuracy: 0.5000 - loss: 0.6932
Epoch 8/30
161/161 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms

In [7]:
confusionMatrix = confusion_matrix(testingYSet, predicted_classes)
print("Confusion Matrix:\n", confusionMatrix)

accuracy = accuracy_score(testingYSet, predicted_classes)
print("Accuracy:\n", accuracy)

recall = recall_score(testingYSet, predicted_classes)
print("Recall:\n", recall)

f1Score = f1_score(testingYSet, predicted_classes)
print("F1 Score:\n", f1Score)

Confusion Matrix:
 [[26 47]
 [14 49]]
Accuracy:
 0.5514705882352942
Recall:
 0.7777777777777778
F1 Score:
 0.6163522012578616


In [8]:
# its kinda weird, but it needs a dataset of representations to quantize to see what to scale immediate activation weights as
def representative_dataset():
    for i in range(810):  # take ~100 samples from your training set
        # x_train[i] shape: (96, 96, 1), values between 0 and 1
        # Must add batch dimension
        yield [trainingXSet[i:i+1].astype('float32')]


# begin converting, not only to tflite but also doing quantization

# specifies what we're gonna convert to
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# give it the function to calculate middle activations
converter.representative_dataset = representative_dataset

converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Force int8 quantization for both weights and activations
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Set input/output to uint8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

In [9]:
# actually make the quantized small model
finalModel = converter.convert()

# now, save it to a file
with open("../alt_cnn_model_uint8.tflite", "wb") as f:
    f.write(finalModel)


INFO:tensorflow:Assets written to: C:\Users\mahd\AppData\Local\Temp\tmpbnmk9cwu\assets


INFO:tensorflow:Assets written to: C:\Users\mahd\AppData\Local\Temp\tmpbnmk9cwu\assets


Saved artifact at 'C:\Users\mahd\AppData\Local\Temp\tmpbnmk9cwu'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2174341337488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2174341338640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2174341337296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2174341336912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2174341336720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2174341338064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2174341336144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2174341338256: TensorSpec(shape=(), dtype=tf.resource, name=None)


c:\Users\mahd\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\lite\python\convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
